# Confusion Matrix

Implements a confusion matrix calculation in PyTorch tests the sklearn, pytorch-cpu, and pytorch-gpu performance on a simple synthetic example. Reference link: https://en.wikipedia.org/wiki/Confusion_matrix


In [ ]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix

In [ ]:
class ConfusionMatrix(torch.nn.Module):
    """Module for calculating confusion matrix w/ PyTorch"""
    def __init__(self):
        super().__init__()

    def calculate_single_channel(self, y_true, y_pred):
        y_true_idx, y_pred_idx = torch.meshgrid(
            y_true.unique(sorted=True), 
            y_pred.unique(sorted=True), 
            indexing="ij"
        )
        conf_mat = torch.zeros_like(y_true_idx)
        for i in range(y_true_idx.shape[0]):
            for j in range(y_true_idx.shape[1]):
                conf_mat[i, j] = torch.sum(
                    torch.logical_and(y_true == y_true_idx[i, j], y_pred == y_pred_idx[i, j])
                )
        return conf_mat

    def forward(self, y_true, y_pred, stack: bool = True):
        """Caclulate the confusion matrix

        Args:
            y_true: true value tensor
            y_pred: predicted value tensor
            stack: bool for whehter or not to attempt to stack the channel-wise results
        Return:
            channel-wise confusion matrix results
        """
        if y_true.dim() > 2:
            if stack:
                return torch.stack([
                    self.calculate_single_channel(t, p) for t, p in zip(y_true, y_pred) 
                ])
            else:
                return [
                    self.calculate_single_channel(t, p) for t, p in zip(y_true, y_pred) 
                ]
        else:
            return self.calculate_single_channel(y_true, y_pred)

In [ ]:
# single multi-class mask
n_classes = 10
y_true = torch.randint(0, n_classes, (512, 512))
y_pred = torch.randint(0, n_classes, (512, 512))

# multiple binary masks
n_classes = 10
y_true = torch.randint(0, 2, (n_classes, 512, 512))
y_pred = torch.randint(0, 2, (n_classes, 512, 512))

y_true_np = y_true.numpy()
y_pred_np = y_pred.numpy()

### Time sklearn

In [ ]:
%%timeit
_ = np.stack([
    confusion_matrix(yt.flatten(), yp.flatten()) for yt, yp in zip(y_true_np, y_pred_np)
])

In [ ]:
cm = ConfusionMatrix()

### Time torch cpu

In [ ]:
%%timeit
_ = cm(y_true, y_pred)

In [ ]:
y_true = y_true.to("cuda")
y_pred = y_pred.to("cuda")

### Time torch gpu

In [ ]:
%%timeit
_ = cm(y_true, y_pred)

In [ ]:
np.allclose(
    np.stack([confusion_matrix(yt.flatten(), yp.flatten()) for yt, yp in zip(y_true_np, y_pred_np)]),
    cm(y_true, y_pred).cpu().numpy()
)